In [616]:
import pandas as pd
import numpy as np
import datetime
import os

from DataTool import DataLoader
data_loader = DataLoader()
analysis_folder_name = 'german_test_1'
baseline_csv_name = 'german-0406.csv'

w_mean   = 0.7
w_std    = 0.3

w_recall = 0.3
w_auc    = 0.7
w_gmean  = 0

best_result_pick_num = 3
pd.set_option('display.float_format',lambda x : '%.5f' % x)

In [617]:
baseline = data_loader.load_csv_to_pandas(f'./result/baseline/{baseline_csv_name}')
results  = data_loader.load_csv_to_pandas(f'./result/{analysis_folder_name}/result.csv')

In [618]:
baseline

,resampler,recall_mean,recall_std,auc_mean,auc_std,gmean_mean,gmean_std,precision_mean,precision_std,fscore_mean,fscore_std
0,MWMOTE,0.56341,0.11509,0.66886,0.03005,0.65409,0.03881,0.49265,0.05106,0.51784,0.04278
1,adasyn,0.51951,0.07116,0.66687,0.03916,0.64901,0.04711,0.51246,0.05192,0.51507,0.05686
2,basic_smote,0.52195,0.07535,0.66854,0.03732,0.65046,0.04644,0.51538,0.04631,0.51697,0.05464
3,bordered_smote,0.52927,0.05460,0.66601,0.02515,0.65061,0.03116,0.50437,0.03648,0.51484,0.03589
4,instance_hardness_threshold,0.83293,0.03638,0.68045,0.02060,0.66273,0.02041,0.39923,0.01632,0.53955,0.01967
5,near_miss,0.69390,0.03325,0.64053,0.03062,0.63728,0.03279,0.38962,0.03161,0.49819,0.02844
6,no_resampling,0.44878,0.08884,0.64274,0.02888,0.60858,0.04520,0.51558,0.05003,0.47353,0.04729
7,one_sided_selection,0.50244,0.08581,0.64847,0.03201,0.62855,0.04302,0.48146,0.03834,0.48814,0.04750
8,random_over_sampling,0.57683,0.08508,0.66410,0.02919,0.65499,0.03683,0.47098,0.04625,0.51397,0.03983
9,random_under_sampling,0.70000,0.02828,0.69587,0.02121,0.69562,0.02120,0.46158,0.02608,0.55599,0.02441


In [619]:
print(results.shape)
# drop columns
drop_columns = ['resample_time', 'train_time', 'support', 'shuffle', 'original_maj_min', 'train_args']
results = results.drop(drop_columns, axis=1)
# results.head(5)

(2250, 17)


In [620]:
#! results先根据id排序
sorted_results = results.sort_values(by='id')
# sorted_results.head(15)

In [621]:
# group by args
grouped_results = sorted_results.groupby(['resample_args']) \
                                .agg(recall_mean=('recall', 'mean'),
                                     recall_std=('recall', 'std'),
                                     auc_mean=('auc', 'mean'),
                                     auc_std=('auc', 'std'),
                                     gmean_mean=('gmean', 'mean'),
                                     gmean_std=('gmean', 'std'),
                                     precision_mean=('precision', 'mean'),
                                     precision_std=('precision', 'std'),
                                     fscore_mean=('fscore', 'mean'),
                                     fscore_std=('fscore', 'std'))
# grouped_results.head(5)

In [622]:
#! add rank info
# grouped_results['recall_mean_rank']  =  grouped_results['recall_mean'].rank(method='dense', ascending=False)
# grouped_results['recall_std_rank']   =  grouped_results['recall_std'].rank(method='dense', ascending=False)
# grouped_results['auc_mean_rank']     =  grouped_results['auc_mean'].rank(method='dense', ascending=False)
# grouped_results['auc_std_rank']      =  grouped_results['auc_std'].rank(method='dense', ascending=False)
# grouped_results['gmean_mean_rank']   =  grouped_results['gmean_mean'].rank(method='dense', ascending=False)
# grouped_results['gmean_std_rank']    =  grouped_results['gmean_std'].rank(method='dense', ascending=False)
grouped_results['recall_mean_norm']  =  (grouped_results['recall_mean'] - grouped_results['recall_mean'].min()) / (grouped_results['recall_mean'].max() - grouped_results['recall_mean'].min())
grouped_results['recall_std_norm']   =  (grouped_results['recall_std']  - grouped_results['recall_std'].max())  / -(grouped_results['recall_std'].max()  - grouped_results['recall_std'].min())
grouped_results['auc_mean_norm']     =  (grouped_results['auc_mean']    - grouped_results['auc_mean'].min())    / (grouped_results['auc_mean'].max()    - grouped_results['auc_mean'].min())
grouped_results['auc_std_norm']      =  (grouped_results['auc_std']     - grouped_results['auc_std'].max())     / -(grouped_results['auc_std'].max()     - grouped_results['auc_std'].min())
grouped_results['gmean_mean_norm']   =  (grouped_results['gmean_mean']  - grouped_results['gmean_mean'].min())  / (grouped_results['gmean_mean'].max()  - grouped_results['gmean_mean'].min())
grouped_results['gmean_std_norm']    =  (grouped_results['gmean_std']   - grouped_results['gmean_std'].max())   / -(grouped_results['gmean_std'].max()   - grouped_results['gmean_std'].min())
grouped_results['score']             =  w_recall * ( w_mean * grouped_results['recall_mean_norm'] + w_std * grouped_results['recall_std_norm']) + \
                                        w_auc    * ( w_mean * grouped_results['auc_mean_norm']    + w_std * grouped_results['auc_std_norm'   ]) + \
                                        w_gmean  * ( w_mean * grouped_results['gmean_mean_norm']  + w_std * grouped_results['gmean_std_norm' ])
grouped_results = grouped_results.sort_values(by='score', ascending=False)
grouped_results.head(best_result_pick_num)

,recall_mean,recall_std,auc_mean,auc_std,gmean_mean,gmean_std,precision_mean,precision_std,fscore_mean,fscore_std,recall_mean_norm,recall_std_norm,auc_mean_norm,auc_std_norm,gmean_mean_norm,gmean_std_norm,score
resample_args,,,,,,,,,,,,,,,,,
"{'a_ros': 0.5, 'a_rus': 1.5, 'i_ros': 0.4}",0.66585,0.06560,0.69095,0.02453,0.68901,0.02552,0.46967,0.02576,0.54941,0.02932,1.00000,0.61220,1.00000,0.61210,1.00000,0.67553,0.88364
"{'a_ros': 2, 'a_rus': 1, 'i_ros': 0.3}",0.64756,0.04281,0.68640,0.01225,0.68438,0.01324,0.47130,0.02050,0.54446,0.01487,0.81481,0.91072,0.85313,0.97451,0.88524,0.98639,0.87576
"{'a_ros': 2.5, 'a_rus': 1.5, 'i_ros': 0.2}",0.65488,0.05046,0.68455,0.01567,0.68247,0.01611,0.46534,0.02694,0.54245,0.01835,0.88889,0.81046,0.79366,0.87352,0.83815,0.91380,0.83194


In [623]:
# slice_col  = ['recall_mean', 'recall_std', 'auc_mean', 'auc_std', 'gmean_mean', 'gmean_std']
best_result = grouped_results.iloc[:best_result_pick_num, 0:10]
best_result['resampler'] = grouped_results.index[:best_result_pick_num]
best_result

,recall_mean,recall_std,auc_mean,auc_std,gmean_mean,gmean_std,precision_mean,precision_std,fscore_mean,fscore_std,resampler
resample_args,,,,,,,,,,,
"{'a_ros': 0.5, 'a_rus': 1.5, 'i_ros': 0.4}",0.66585,0.06560,0.69095,0.02453,0.68901,0.02552,0.46967,0.02576,0.54941,0.02932,"{'a_ros': 0.5, 'a_rus': 1.5, 'i_ros': 0.4}"
"{'a_ros': 2, 'a_rus': 1, 'i_ros': 0.3}",0.64756,0.04281,0.68640,0.01225,0.68438,0.01324,0.47130,0.02050,0.54446,0.01487,"{'a_ros': 2, 'a_rus': 1, 'i_ros': 0.3}"
"{'a_ros': 2.5, 'a_rus': 1.5, 'i_ros': 0.2}",0.65488,0.05046,0.68455,0.01567,0.68247,0.01611,0.46534,0.02694,0.54245,0.01835,"{'a_ros': 2.5, 'a_rus': 1.5, 'i_ros': 0.2}"


In [624]:
baseline = pd.concat([baseline, best_result], axis=0)
# baseline

In [625]:
baseline['recall_mean_norm']  =  (baseline['recall_mean'] - baseline['recall_mean'].min()) / (baseline['recall_mean'].max() - baseline['recall_mean'].min())
baseline['recall_std_norm']   =  (baseline['recall_std']  - baseline['recall_std'].max())  / -(baseline['recall_std'].max()  - baseline['recall_std'].min())
baseline['auc_mean_norm']     =  (baseline['auc_mean']    - baseline['auc_mean'].min())    / (baseline['auc_mean'].max()    - baseline['auc_mean'].min())
baseline['auc_std_norm']      =  (baseline['auc_std']     - baseline['auc_std'].max())     / -(baseline['auc_std'].max()     - baseline['auc_std'].min())
baseline['gmean_mean_norm']   =  (baseline['gmean_mean']  - baseline['gmean_mean'].min())  / (baseline['gmean_mean'].max()  - baseline['gmean_mean'].min())
baseline['gmean_std_norm']    =  (baseline['gmean_std']   - baseline['gmean_std'].max())   / -(baseline['gmean_std'].max()   - baseline['gmean_std'].min())
baseline['score']             =  w_recall * ( w_mean * baseline['recall_mean_norm']  + w_std * baseline['recall_std_norm']) + \
                                 w_auc    * ( w_mean * baseline['auc_mean_norm']     + w_std * baseline['auc_std_norm'   ]) + \
                                 w_gmean  * ( w_mean * baseline['gmean_mean_norm']   + w_std * baseline['gmean_std_norm' ])
baseline = baseline.sort_values(by='score', ascending=False)
baseline.head()

,resampler,recall_mean,recall_std,auc_mean,auc_std,gmean_mean,gmean_std,precision_mean,precision_std,fscore_mean,fscore_std,recall_mean_norm,recall_std_norm,auc_mean_norm,auc_std_norm,gmean_mean_norm,gmean_std_norm,score
9,random_under_sampling,0.70000,0.02828,0.69587,0.02121,0.69562,0.02120,0.46158,0.02608,0.55599,0.02441,0.65397,1.00000,1.00000,0.66704,1.00000,0.77209,0.85741
"{'a_ros': 2, 'a_rus': 1, 'i_ros': 0.3}","{'a_ros': 2, 'a_rus': 1, 'i_ros': 0.3}",0.64756,0.04281,0.68640,0.01225,0.68438,0.01324,0.47130,0.02050,0.54446,0.01487,0.51746,0.83265,0.82877,1.00000,0.87082,1.00000,0.79970
4,instance_hardness_threshold,0.83293,0.03638,0.68045,0.02060,0.66273,0.02041,0.39923,0.01632,0.53955,0.01967,1.00000,0.90668,0.72142,0.68977,0.62213,0.79480,0.78995
"{'a_ros': 2.5, 'a_rus': 1.5, 'i_ros': 0.2}","{'a_ros': 2.5, 'a_rus': 1.5, 'i_ros': 0.2}",0.65488,0.05046,0.68455,0.01567,0.68247,0.01611,0.46534,0.02694,0.54245,0.01835,0.53651,0.74448,0.79541,0.87290,0.84898,0.91793,0.75273
"{'a_ros': 0.5, 'a_rus': 1.5, 'i_ros': 0.4}","{'a_ros': 0.5, 'a_rus': 1.5, 'i_ros': 0.4}",0.66585,0.06560,0.69095,0.02453,0.68901,0.02552,0.46967,0.02576,0.54941,0.02932,0.56508,0.57014,0.91115,0.54389,0.92405,0.64856,0.73066
